# セッション3-B: Cortex Agent の設定

このノートブックでは、Cortex Agentを作成し、AIアシスタントがサステナビリティレポートを分析できるようにします。

## 前提条件
- セッション3-A（Cortex Search）が完了していること
- `sustainability_report` Cortex Search Serviceが作成されていること

## このセッションで学ぶこと
1. **Cortex Agent**の概念と構造
2. Agentの作成（YAML仕様）
3. Agentのテスト


## 📦 接続設定


In [ ]:
-- 接続情報の設定
USE ROLE ACCOUNTADMIN;
USE DATABASE DEMO_DB;
USE SCHEMA DEMO_SUSTAINABILITY;
USE WAREHOUSE HANDSON_WH;


## 🤖 Cortex Agentとは？

**Cortex Agent**は、Snowflake上でAIエージェントを構築するためのサービスです。
LLM（大規模言語モデル）と外部ツール（Cortex Search、Cortex Analystなど）を組み合わせて、複雑なタスクを自動化できます。

### 主な特徴
- **ツール統合**: Cortex Search、Cortex Analyst、カスタムツールと連携
- **会話履歴**: マルチターンの会話をサポート
- **YAML仕様**: 宣言的な設定でAgentを定義
- **トークン/時間制限**: バジェット管理で予期せぬコストを防止

### Agent仕様の構造

```yaml
models:
  orchestration: claude-4-sonnet  # 使用するLLMモデル

orchestration:
  budget:
    seconds: 60      # 最大実行時間
    tokens: 32000    # 最大トークン数

instructions:
  system: |          # システムプロンプト（Agentの役割）
    ...
  response: |        # 応答のフォーマット指示
    ...
  orchestration: |   # ツール使用の指示
    ...

tools:               # 使用するツール
  - tool_spec:
      type: "cortex_search"
      name: "ツール名"
      description: "ツールの説明"

tool_resources:      # ツールのリソース設定
  ツール名:
    name: "Cortex Search Service名"
    max_results: 10
```


---
## Step 3-3: Cortex Agent の作成

スチュワードシップ原則評価用のAIエージェントを作成します。

### このAgentの機能
- GPIFスチュワードシップ活動原則に基づく評価
- 運用機関のサステナビリティレポートの分析
- 5つの原則への対応状況の評価

**参考ドキュメント**: https://docs.snowflake.com/en/sql-reference/sql/create-agent


In [ ]:
-- スチュワードシップ原則評価用のAIエージェント
CREATE OR REPLACE AGENT STEWARDSHIP_AGENT
    COMMENT = 'GPIFスチュワードシップ活動原則に基づき、運用機関のサステナビリティレポートを分析・評価するエージェント'
    PROFILE = '{"display_name": "スチュワードシップ評価アシスタント", "color": "blue"}'
    FROM SPECIFICATION
$$
models:
  orchestration: claude-4-sonnet

orchestration:
  budget:
    seconds: 60
    tokens: 32000

instructions:
  system: |
    あなたはスチュワードシップ活動原則の専門家です。
    運用機関のサステナビリティレポートを分析し、5つの原則への対応状況を評価します。
    
    【スチュワードシップ活動原則】
    原則1: 運用受託機関におけるコーポレート・ガバナンス体制
    原則2: 運用受託機関における利益相反管理
    原則3: エンゲージメントを含むスチュワードシップ活動方針
    原則4: 投資におけるESGなどのサステナビリティの考慮
    原則5: 議決権行使

  response: |
    回答は日本語で行い、必ず出典（ファイル名、ページ番号）を明記してください。
    原則の趣旨に沿った取り組みが確認できれば「対応している」と評価してください。
    情報が見つからない場合のみ「情報なし」と回答してください。

  orchestration: |
    運用機関のレポートに関する質問にはSustainabilitySearchツールを使用してください。
    検索結果に基づいて回答を構成してください。

  sample_questions:
    - question: "AMOneのスチュワードシップ責任に関する方針を教えてください"
      answer: "検索ツールを使用してAMOneのレポートから情報を取得します。"
    - question: "各運用機関の利益相反管理の取り組みを比較してください"
      answer: "全運用機関のレポートを検索し、利益相反管理に関する情報を収集して比較します。"

tools:
  - tool_spec:
      type: "cortex_search"
      name: "SustainabilitySearch"
      description: "運用機関のサステナビリティレポートとGPIFスチュワードシップ活動原則を検索します"

tool_resources:
  SustainabilitySearch:
    name: "DEMO_DB.DEMO_SUSTAINABILITY.SUSTAINABILITY_REPORT"
    max_results: 12
    title_column: "RELATIVE_PATH"
    id_column: "SCOPED_FILE_URL"
$$;


In [ ]:
-- Agent確認
SHOW AGENTS;


---
## 📊 動作確認

作成したテーブル、ビュー、サービスを確認します。


In [ ]:
-- データ件数の確認
SELECT 'am_sustainability_report_chunk' AS table_name, COUNT(*) AS row_count FROM am_sustainability_report_chunk
UNION ALL
SELECT 'gpif_stewardship_2025_chunk', COUNT(*) FROM gpif_stewardship_2025_chunk
UNION ALL
SELECT 'gpif_sustainability_report_chunk', COUNT(*) FROM gpif_sustainability_report_chunk
UNION ALL
SELECT 'global_pf_sustainability_report_chunk', COUNT(*) FROM global_pf_sustainability_report_chunk;


In [ ]:
-- AI_COMPLETE動作確認
SELECT AI_COMPLETE(
    'claude-sonnet-4-5',
    'こんにちは。Snowflake Cortex AIのテストです。'
) AS test_response;


---
## 🎉 ハンズオン完了！

### 作成したリソースまとめ

| カテゴリ | リソース名 | 説明 |
|---------|-----------|------|
| **テーブル** | `am_sustainability_report` | 運用機関レポートのテキスト |
| | `am_sustainability_report_chunk` | 運用機関レポートのチャンク |
| | `gpif_stewardship_2025` | スチュワードシップ原則のテキスト |
| | `gpif_stewardship_2025_chunk` | スチュワードシップ原則のチャンク |
| | `gpif_sustainability_report` | GPIF ESGレポートのテキスト |
| | `gpif_sustainability_report_chunk` | GPIF ESGレポートのチャンク |
| | `global_pf_sustainability_report` | 海外年金基金レポートのテキスト |
| | `global_pf_sustainability_report_chunk` | 海外年金基金レポートのチャンク |
| **ビュー** | `combined_sustainability_chunks_view` | スチュワードシップ評価用統合ビュー |
| | `combined_global_sustainability_view` | グローバル分析用統合ビュー |
| **Cortex Search** | `sustainability_report` | スチュワードシップ評価用検索 |
| | `global_pf_sustainability_report` | グローバル分析用検索 |
| **Cortex Agent** | `STEWARDSHIP_AGENT` | スチュワードシップ評価エージェント |

### 次のステップ
- Streamlitアプリでこれらのリソースを活用
- Cortex Agentとの対話によるレポート分析
